In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [49]:
df = pd.read_csv('D:/Professional/data/questionPair/train.csv')

In [50]:
df.shape

(404290, 6)

In [51]:
new_df = df.sample(30000)

In [52]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30000 entries, 181260 to 297122
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            30000 non-null  int64 
 1   qid1          30000 non-null  int64 
 2   qid2          30000 non-null  int64 
 3   question1     30000 non-null  object
 4   question2     30000 non-null  object
 5   is_duplicate  30000 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 1.6+ MB


In [53]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [54]:
new_df.duplicated().sum()

0

In [55]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
181260,"How would you know if a guy really likes you, ...",How do I know if a guy really means what he says?
184743,What are the best halal restaurants in Toronto?,How many restaurants are there in Toronto?
266563,Why is the scholarly consensus that Jesus of N...,What is the best evidence for a historical Jesus?
379855,What is the corporate culture like at US Silic...,What is the corporate culture like at US Cellu...
329197,How is bitumen manufactured?,Can bitumen be imported in India?


# Vectorization

In [56]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [57]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(30000, 6000)

In [58]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
181260,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
184743,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
266563,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
379855,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
329197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242218,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55398,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
245832,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
94005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [59]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [60]:
temp_df.columns

Index([             0,              1,              2,              3,
                    4,              5,              6,              7,
                    8,              9,
       ...
                 2991,           2992,           2993,           2994,
                 2995,           2996,           2997,           2998,
                 2999, 'is_duplicate'],
      dtype='object', length=6001)

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1)

In [62]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [63]:
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7451666666666666

In [69]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.7241666666666666